 # Data Cleaning — E-Commerce Customer Behavior

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)



In [2]:
df = pd.read_csv("../data/ecommerece-backup.csv")

In [3]:
#set visulization style 

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12,6)
print("libraries loaded ")

libraries loaded 


# Data Inspection 

In [5]:
# Checking the missing values 

missing_summary = pd.DataFrame({ 
     "column":df.columns,
     "Missings": df.isnull().sum(),
     "percentages" :(df.isnull().sum() / len(df) *100).round(2)
})
missing_summary = missing_summary[missing_summary["Missings"]>0].sort_values("Missings",ascending=False)

if len(missing_summary) >0 :
     print(missing_summary.to_string(index=False))
     print(f" Total column with missing values {missing_summary}")
else:
     print("No missing values found")
print()

# Checking the duplicate rows 

duplicates = df.duplicated().sum()
print(f"Duplicated values found in row {duplicates}")
if duplicates > 0:
     print(f"percentage :{(duplicates/len(df)*100)}:.2f%")
     print("\n first few duplicates")
     print(df[df.duplicated(keep=False)].head())
else:
     print("No duplicated found ")
print()

# 3. Data Types Check

print(df.dtypes)
print()

# 4. Basic Stats for Numerical Columns (to spot issues)
numerical_cols = df.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
     print(f"\n{col}:")
     print(f"Min:{df[col].min()}")
     print(f"Max:{df[col].max()}")
     print(f"Mean:{df[col].mean():.2f}")
     print(f"Median:{df[col].median():.2f}")

# Check for potential issues
     if df[col].min() < 0 and col in ['Age', 'Price', 'Quantity']:
          print(f"WARNING: Negative values found (shouldn't be negative)")
     if df[col].max() > 1000 and df[col].median() < 100:
          print(f"NOTICE: Very large outliers detected")

print()


No missing values found

Duplicated values found in row 0
No duplicated found 

Unnamed: 0                    int64
customer_id                   int64
gender                       object
age                           int64
city                         object
membership_type              object
total_spend                 float64
items_purchased               int64
average_rating              float64
discount_applied               bool
days_since_last_purchase      int64
satisfaction_level           object
dtype: object


Unnamed: 0:
Min:0
Max:349
Mean:174.89
Median:175.50

customer_id:
Min:101
Max:450
Mean:275.89
Median:276.50

age:
Min:26
Max:43
Mean:33.58
Median:32.00

total_spend:
Min:410.8
Max:1520.1
Mean:847.79
Median:780.20

items_purchased:
Min:7
Max:21
Mean:12.63
Median:12.00

average_rating:
Min:3.0
Max:4.9
Mean:4.02
Median:4.10

days_since_last_purchase:
Min:9
Max:63
Mean:26.61
Median:23.00



In [11]:
# Removing the unnecessary column
# Check if 'Unnamed: 0' exists (it's a redundant index column)
if 'Unnamed: 0' in df.columns:
     print("Before columns:", list(df.columns))
     print(f"Total columns: {len(df.columns)}")
     print()
     
     # Drop the column
     df = df.drop(columns=['Unnamed: 0'])
     
     print("Removed 'Unnamed: 0' column")
     print()
     print("After columns:", list(df.columns))
     print(f"Total columns: {len(df.columns)}")
else:
     print("No unnecessary columns found")

print()

Before columns: ['Unnamed: 0', 'customer_id', 'gender', 'age', 'city', 'membership_type', 'total_spend', 'items_purchased', 'average_rating', 'discount_applied', 'days_since_last_purchase', 'satisfaction_level']
Total columns: 12

Removed 'Unnamed: 0' column

After columns: ['customer_id', 'gender', 'age', 'city', 'membership_type', 'total_spend', 'items_purchased', 'average_rating', 'discount_applied', 'days_since_last_purchase', 'satisfaction_level']
Total columns: 11



# Fill with mean/Median 

In [7]:
numerical_cols = df.select_dtypes(include=[np.number]).columns
missing_numerical = [col for col in numerical_cols if df[col].isnull().sum() > 0]

if len(missing_numerical) > 0:
     print(f"Found {len(missing_numerical)} numerical column(s) with missing values:")
     print()
     
     for col in missing_numerical:
          missing_count = df[col].isnull().sum()
          mean_val = df[col].mean()
          median_val = df[col].median()
          
          print(f"{col}:")
          print(f"Missing:{missing_count} ({(missing_count/len(df)*100):.1f}%)")
          print(f"Mean:{mean_val:.2f}")
          print(f" Median:{median_val:.2f}")
          
          # Recommendation
          if abs(mean_val - median_val) / median_val > 0.1:  # More than 10% difference
               print(f"Recommendation: Use MEDIAN (data is skewed)")
               recommended = "median"
          else:
               print(f"Recommendation: Use MEAN (data is normal)")
               recommended = "mean"
               print()
else:

     print("No missing values in numerical columns!")

No missing values in numerical columns!


In [12]:
# ============================================================
# STEP 3: OUTLIER CHECK (VERIFICATION)
# ============================================================

print("=" * 70)
print("STEP 3: OUTLIER DETECTION — IQR METHOD")
print("=" * 70)
print()

numerical_cols = ['age', 'total_spend', 'items_purchased', 
                  'average_rating', 'days_since_last_purchase']

outlier_summary = []

for col in numerical_cols:
    # Calculate IQR
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Find outliers
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    
    print(f"📊 {col}:")
    print(f"   Q1 (25%):      {Q1:.2f}")
    print(f"   Q3 (75%):      {Q3:.2f}")
    print(f"   IQR:           {IQR:.2f}")
    print(f"   Lower fence:   {lower_bound:.2f}")
    print(f"   Upper fence:   {upper_bound:.2f}")
    print(f"   Outliers:      {len(outliers)} ({(len(outliers)/len(df)*100):.1f}%)")
    
    if len(outliers) > 0:
        print(f"   Outlier range: {outliers[col].min():.2f} to {outliers[col].max():.2f}")
        print(f"   ⚠️  Outliers detected")
        
        outlier_summary.append({
            'Column': col,
            'Count': len(outliers),
            'Percentage': f"{(len(outliers)/len(df)*100):.1f}%"
        })
    else:
        print(f"   ✅ No outliers")
    
    print()

# Summary
if len(outlier_summary) > 0:
    print("=" * 70)
    print("OUTLIER SUMMARY:")
    print("-" * 70)
    outlier_df = pd.DataFrame(outlier_summary)
    print(outlier_df.to_string(index=False))
    print()
    print("💡 RECOMMENDATION:")
    print("   Check if outliers are:")
    print("   - Real extreme values (VIP customers) → KEEP them")
    print("   - Data entry errors → REMOVE them")
else:
    print("=" * 70)
    print("✅ NO OUTLIERS DETECTED — DATA IS CLEAN!")

print()
print("=" * 70)

STEP 3: OUTLIER DETECTION — IQR METHOD

📊 age:
   Q1 (25%):      30.00
   Q3 (75%):      37.00
   IQR:           7.00
   Lower fence:   19.50
   Upper fence:   47.50
   Outliers:      0 (0.0%)
   ✅ No outliers

📊 total_spend:
   Q1 (25%):      505.75
   Q3 (75%):      1160.60
   IQR:           654.85
   Lower fence:   -476.52
   Upper fence:   2142.88
   Outliers:      0 (0.0%)
   ✅ No outliers

📊 items_purchased:
   Q1 (25%):      9.00
   Q3 (75%):      15.00
   IQR:           6.00
   Lower fence:   0.00
   Upper fence:   24.00
   Outliers:      0 (0.0%)
   ✅ No outliers

📊 average_rating:
   Q1 (25%):      3.50
   Q3 (75%):      4.50
   IQR:           1.00
   Lower fence:   2.00
   Upper fence:   6.00
   Outliers:      0 (0.0%)
   ✅ No outliers

📊 days_since_last_purchase:
   Q1 (25%):      15.00
   Q3 (75%):      38.00
   IQR:           23.00
   Lower fence:   -19.50
   Upper fence:   72.50
   Outliers:      0 (0.0%)
   ✅ No outliers

✅ NO OUTLIERS DETECTED — DATA IS CLEAN!



# Handaling Outliers 

In [10]:
# ============================================================
# CLEANING: HANDLE OUTLIERS
# ============================================================

print("=" * 70)
print("OUTLIER TREATMENT")
print("=" * 70)
print()

# Choose numerical columns to check (replace with your column names)
# Example: If you found outliers in 'Total_Spend', 'Age', etc.

numerical_cols = df.select_dtypes(include=[np.number]).columns

print("Choose your strategy for each column with outliers:")
print()

for col in numerical_cols:
    # Calculate IQR
    Q1 = df[col].quantile(0.25)  # this is finding 25%
    Q3 = df[col].quantile(0.75)  # this is finding 75%
    IQR = Q3 - Q1   #actual IQR 
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Find outliers
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    
    if len(outliers) > 0:
        print(f"{col}:")
        print(f"Outliers found: {len(outliers)} ({(len(outliers)/len(df)*100):.1f}%)")
        print(f"Normal range: {lower_bound:.2f} to {upper_bound:.2f}")
        print(f"Outlier range: {outliers[col].min():.2f} to {outliers[col].max():.2f}")
        print()
        
        # print("Choose a strategy:")
        # print("      1. KEEP — If they're real values (VIP customers, special cases)")
        # print("      2. REMOVE — If they're data errors")
        # print("      3. CAP — Replace extreme values with the fence values")
        # print()
        
        # STRATEGY 1: Keep (do nothing)
        # No code needed
        
        # STRATEGY 2: Remove outliers
        # UNCOMMENT to use:
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
        print(f"   ✅ Removed {len(outliers)} outliers from {col}")
        
        # STRATEGY 3: Cap outliers (Winsorization)
        # UNCOMMENT to use:
        # df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
        # print(f"   ✅ Capped outliers in {col} to range {lower_bound:.2f} - {upper_bound:.2f}")
        


print()

OUTLIER TREATMENT

Choose your strategy for each column with outliers:




# Final Validation

In [13]:
# ============================================================
# STEP 5: FINAL VALIDATION
# ============================================================

print("=" * 70)
print("🎯 FINAL DATA QUALITY CHECK")
print("=" * 70)
print()

print("📊 Dataset Shape:")
print(f"   Rows:    {len(df):,}")
print(f"   Columns: {len(df.columns)}")
print()

print("🔍 Data Quality:")
print(f"   Missing values:   {df.isnull().sum().sum()}")
print(f"   Duplicate rows:   {df.duplicated().sum()}")
print(f"   Memory usage:     {df.memory_usage(deep=True).sum() / 1024:.2f} KB")
print()

print("📋 Column List:")
print(df.columns.tolist())
print()

print("📈 Data Types:")
print(df.dtypes.value_counts())
print()

print("✅ CLEANING COMPLETE!")
print()
print("=" * 70)

🎯 FINAL DATA QUALITY CHECK

📊 Dataset Shape:
   Rows:    348
   Columns: 11

🔍 Data Quality:
   Missing values:   0
   Duplicate rows:   0
   Memory usage:     92.87 KB

📋 Column List:
['customer_id', 'gender', 'age', 'city', 'membership_type', 'total_spend', 'items_purchased', 'average_rating', 'discount_applied', 'days_since_last_purchase', 'satisfaction_level']

📈 Data Types:
int64      4
object     4
float64    2
bool       1
Name: count, dtype: int64

✅ CLEANING COMPLETE!

